# Nurse Scheduling Problem

Lucerne University of Applied Sciences and Arts - School of Information Technology

@author: Tobias Mérinat

Imports

In [63]:
from ortools.constraint_solver import pywrapcp
from itertools import product

Pretty-print schedule

In [64]:
def pretty_print(schedule):
    mapper1 = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
    mapper2 = {0: 'HO |', 1: 'S1 |', 2: 'S2 |', 3: 'S3 |'}
    print("SHIFT | M T W T F S S")
    print("---------------------")
    for i in range(len(schedule)):
        print("   {} ".format(mapper2[i]), end='')
        for j in range(len(schedule[0])):
            print("{} ".format(mapper1[schedule[i][j].Value()]), end='')
        print("")
    print("\n\n")

Nurse Schedule Problem from lecture

In [65]:
num_nurses = 4
num_days = 7
holiday_shift = 0
day_shift = 1
late_shift = 2
night_shift = 3
shifts = (holiday_shift, day_shift, late_shift, night_shift)

Create constraint solver

In [66]:
solver = pywrapcp.Solver("Nurses")

Schedule configuration

In [67]:
num_shifts = len(shifts)
schedule_indices = list(product(range(num_shifts), range(num_days)))  # tuples (i, j) for all schedule indices

Decision variables: schedule[s][d]==n means nurse n works shift s on day d

In [68]:
schedule = [[solver.IntVar(0, num_nurses-1) for _d in range(num_days)] for _s in range(num_shifts)]

A Nurse must not work two shifts on the same Day

In [69]:
for day in range(num_days):
    solver.Add(solver.AllDifferent([schedule[shift][day] for shift in range(num_shifts)]))

How many days does nurse k work in shift i (k x i)

In [70]:
days = []
for nurse in range(num_nurses):
    days.append([])
    for shift in range(num_shifts):
        days[nurse].extend([solver.Sum([solver.IsEqualCstVar(schedule[shift][day], nurse) for day in range(num_days)])])

Each nurse has one or two days off

In [71]:
for nurse in range(num_nurses):
    solver.Add(solver.BetweenCt(days[nurse][0], 1, 2))

# Alternative implementation without additional API elements
#for nurse in range(num_nurses):
#    solver.Add(days[nurse][0] > 0);
#    solver.Add(days[nurse][0] < 3);

Each shift is staffed by two different nurses in a week (holiday does not count as shift)

In [72]:
for shift in range(1, num_shifts):
    count = [days[nurse][shift] > 0 for nurse in range(num_nurses)]
    solver.Add(solver.Sum(count) <= 2)

If a nurse works shifts 2 or 3 on a given day, must work the same shift either the previous day or the day after

In [73]:
# This is not gonna win a best coding style award. 
# However, it is much more important to us that stuidents understand the model rather than writing compact code.
# Exercise: you may want to try packing this into a loop
        
solver.Add(solver.Max(schedule[2][0] == schedule[2][1], schedule[2][1] == schedule[2][2]) == 1)
solver.Add(solver.Max(schedule[2][1] == schedule[2][2], schedule[2][2] == schedule[2][3]) == 1)
solver.Add(solver.Max(schedule[2][2] == schedule[2][3], schedule[2][3] == schedule[2][4]) == 1)
solver.Add(solver.Max(schedule[2][3] == schedule[2][4], schedule[2][4] == schedule[2][5]) == 1)
solver.Add(solver.Max(schedule[2][4] == schedule[2][5], schedule[2][5] == schedule[2][6]) == 1)
solver.Add(solver.Max(schedule[2][5] == schedule[2][6], schedule[2][6] == schedule[2][0]) == 1)
solver.Add(solver.Max(schedule[2][6] == schedule[2][0], schedule[2][0] == schedule[2][1]) == 1)

solver.Add(solver.Max(schedule[3][0] == schedule[3][1], schedule[3][1] == schedule[3][2]) == 1)
solver.Add(solver.Max(schedule[3][1] == schedule[3][2], schedule[3][2] == schedule[3][3]) == 1)
solver.Add(solver.Max(schedule[3][2] == schedule[3][3], schedule[3][3] == schedule[3][4]) == 1)
solver.Add(solver.Max(schedule[3][3] == schedule[3][4], schedule[3][4] == schedule[3][5]) == 1)
solver.Add(solver.Max(schedule[3][4] == schedule[3][5], schedule[3][5] == schedule[3][6]) == 1)
solver.Add(solver.Max(schedule[3][5] == schedule[3][6], schedule[3][6] == schedule[3][0]) == 1)
solver.Add(solver.Max(schedule[3][6] == schedule[3][0], schedule[3][0] == schedule[3][1]) == 1)

Configure solver

In [74]:
db = solver.Phase([schedule[i][j] for i, j in schedule_indices], solver.INT_VAR_SIMPLE, solver.INT_VALUE_SIMPLE)

Start solver

In [75]:
solver.NewSearch(db)
while solver.NextSolution():
    # This creates a hell lot of printing
    # pretty_print(schedule)
    ;

Cleanup

In [76]:
solver.EndSearch()

Print solver information

In [77]:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime:   {}ms".format(solver.WallTime()))
print("Failures:  {}".format(solver.Failures()))
print("Branches:  {} ".format(solver.Branches()))

Solutions: 18144
Runtime:   3067ms
Failures:  339120
Branches:  714526 
